In [53]:
from transformers import pipeline
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_openai import OpenAIEmbeddings #for converting chunks into embeddings
from langchain_chroma import Chroma #database for stroring the embeddings
from langchain_huggingface import HuggingFacePipeline

from langchain_community.document_loaders import ImageCaptionLoader

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.retrievers import EnsembleRetriever
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import LLMChain
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from openai import OpenAI

import warnings
import pandas as pd
import tempfile
import shutil
import os
# Dictionary to track temporary directories
temp_dirs = []

In [54]:
def img2doc(img_path):
    
    # Create an ImageCaptionLoader instance
    loader = ImageCaptionLoader(img_path)
    # Load the caption as a document
    documents = loader.load()
    
    # Split the document into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    # Create a temporary directory
    temp_dir = tempfile.mkdtemp()
    temp_dirs.append(temp_dir)  # Track temp directory for later cleanup
    
    # Create a new Chroma vectorstore
    vectorstore = Chroma.from_documents(
        documents=splits, 
        embedding=OpenAIEmbeddings(), 
        persist_directory=temp_dir
    )
    retriever = vectorstore.as_retriever(k=1)
    
    return retriever, temp_dir

In [55]:
def fashiondb(dir):
    """
    dir is the directory of the vector DB
    """
    embeddings_used = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorDB = Chroma(persist_directory=dir,embedding_function=embeddings_used)
    retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    return retriever

In [56]:
def cleanup_temp_dir(temp_dir):
    try:
        shutil.rmtree(temp_dir)
        #print(f"Temporary directory {temp_dir} cleaned up successfully.")
    except Exception as e:
        print(f"Failed to clean up temporary directory {temp_dir}: {e}")

In [57]:
import os
dir = os.getcwd()
db_dir = os.path.join(dir,"chroma_db")
print(db_dir)

c:\Users\admin\OneDrive\Desktop\final\closet_care_buddy-main\chroma_db


In [58]:

def textGeneration_langChain_RAG(img_path):
    llm = ChatOpenAI(model="gpt-4o", temperature=0)


    system_prompt = (
        "You are an experienced clothing sylist. "
        "Use the following pieces of retrieved context to answer. "
        "Describe the piece of clothing in this picture. "
        "Use two sentence maximum and be as detailed as possible yet concise. "
        "Include the clothing syle (i.e. bohemian, casual, classic, sporty, preppy). "
        "Include how new or worn the item looks to be. "
        "Be confident avoid using the word 'likely'. "
        "\n\n"
        "{context}"
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    caption_retriever,temp_dir = img2doc(img_path)
    #caption_retriever = fashiondb(db_dir)

    # initialize the ensemble retriever
    #ensemble_retriever = EnsembleRetriever(
    #retrievers=[caption_retriever, reference_retriever], weights=[0.8, 0.2]
    #)

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(caption_retriever, question_answer_chain)

    response = rag_chain.invoke({"input": "Describe the piece of clothing in this picture", "context": caption_retriever})

    #rag_chain = prompt | llm | StrOutputParser()
    cleanup_temp_dir(temp_dir)

    return response["answer"]

In [59]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env


True

In [60]:
# from dotenv import load_dotenv
# import os
# from langchain_openai import ChatOpenAI

# # Load .env variables
# load_dotenv()

# # Retrieve API key from environment
# api_key = os.getenv("OPENAI_API_KEY")

# # Initialize ChatOpenAI
# llm = ChatOpenAI(api_key=api_key, model="gpt-4o", temperature=0)

# res = textGeneration_langChain_RAG("static/imgs/last_capture_demo.png")
# print(res)


In [61]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# Load .env variables
load_dotenv()

# Retrieve API key from environment
api_key = os.getenv("OPENAI_API_KEY")

# Ensure that the API key is loaded properly
if not api_key:
    print("API key not found. Please check your .env file.")
else:
    # Initialize ChatOpenAI
    llm = ChatOpenAI(api_key=api_key, model="gpt-4o", temperature=0)

    # Assuming textGeneration_langChain_RAG is defined elsewhere
    res = textGeneration_langChain_RAG("static/imgs/sari.png")
    print(res)


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Failed to clean up temporary directory C:\Users\admin\AppData\Local\Temp\tmpfirmekhx: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\admin\\AppData\\Local\\Temp\\tmpfirmekhx\\308938ce-2f02-4f43-8832-591075bcfd78\\data_level0.bin'
The piece of clothing is a vibrant green and yellow sari, showcasing a traditional and elegant style. The fabric appears fresh and well-maintained, indicating it is relatively new.


In [28]:
client = OpenAI()

response = client.images.generate(
    model="dall-e-3",
    prompt="generate_picture_prompt()",
    size="1024x1024",
    quality="standard",
    n=1,
  )

image_url = response.data[0].url
print(image_url)


NameError: name 'OpenAI' is not defined